In [6]:
import pandas as pd
import numpy as np

In [7]:
imdb = pd.read_csv('https://raw.githubusercontent.com/shafayetShafee/HOMLR/main/NaiveBayes/imdb_dataset.csv')

In [8]:
imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
imdb.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [10]:
df = imdb.sample(10_000)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 18092 to 31898
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     10000 non-null  object
 1   sentiment  10000 non-null  object
dtypes: object(2)
memory usage: 234.4+ KB


In [12]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 18092 to 31898
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     10000 non-null  object
 1   sentiment  10000 non-null  object
dtypes: object(2)
memory usage: 13.7 MB


In [13]:
df = df.reset_index(drop=True)

In [14]:
df.head()

,review,sentiment
0,For those of you looking for the crazy stunts ...,positive
1,I and a friend rented this movie. We both foun...,negative
2,I caught this a few months ago on Family Chann...,negative
3,This is one of my favourite films; a delightfu...,positive
4,The sequel that no one asked for to the movie ...,negative


In [15]:
import re
# re.sub('<.*?>', '', df.review[0])

def clean_html(text):
    return re.sub('<.*?>', '', text)

df.review = df.review.apply(clean_html)

In [16]:
df.review[0]

"For those of you looking for the crazy stunts that typified a Harold Lloyd silent comedy, this is not the film for you. What The Cat's-Paw gives us is an interesting and atypical character for Lloyd who was trying to establish himself in sound.For me the closest movie comparison to Lloyd's character is that of Peter Sellers in Being There. For all the education that Lloyd has received in dealing with the world, he might as well have been brought up in isolation as Sellers was.But where he was brought up was as a missionary's child in China and I don't know how much Christianity he and his family were able to teach the Chinese, but young Harold has learned the wisdom of Chinese philosopher Lin Po whom he quotes constantly like a fortune cookie aphorism. As it turns out Lin Po turns out to be one wise dude.Anyway Lloyd's father Samuel S. Hinds has decided his son needs some education in the modern world of 20th century America and he sends him back to be the guest of the pastor of the h

In [17]:
df.review = df.review.apply(lambda x: x.lower())

In [18]:
df.head()

,review,sentiment
0,for those of you looking for the crazy stunts ...,positive
1,i and a friend rented this movie. we both foun...,negative
2,i caught this a few months ago on family chann...,negative
3,this is one of my favourite films; a delightfu...,positive
4,the sequel that no one asked for to the movie ...,negative


In [19]:
df.review = df.review.apply(lambda x: re.sub("[^A-Z\w\d]", " ", x))

In [20]:
import nltk
from nltk.corpus import stopwords

In [21]:
from collections import Counter
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [22]:
def rm_stopwords(text: str) -> str:
    """
    Removes the stopwords (stopword from ntlk library)
    from the given text.
    """
    stop_words_dict = Counter(stop_words)
    return " ".join([word for word in text.split() if word not in stop_words_dict])

In [23]:
df.review

0       for those of you looking for the crazy stunts ...
1       i and a friend rented this movie  we both foun...
2       i caught this a few months ago on family chann...
3       this is one of my favourite films  a delightfu...
4       the sequel that no one asked for to the movie ...
                              ...                        
9995    a mock documentary about a pair of canadian pr...
9996    a prison cell four prisoners carrere a young c...
9997    growing up in the late 60s and 70s i could not...
9998    head should have been a proud moment not just ...
9999     mild spoilers in this would be satire  chapli...
Name: review, Length: 10000, dtype: object

In [24]:
df.review = df.review.apply(rm_stopwords)

In [25]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [26]:
def stem_words(text: str) -> str:
    stemmed = []
    for word in text.split():
        stemmed.append(ps.stem(word))
    return " ".join(stemmed)

In [27]:
df.review = df.review.apply(stem_words)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
cnv = CountVectorizer()

In [29]:
X = cnv.fit_transform(df.review).toarray()

In [30]:
X.shape

(10000, 36350)

In [31]:
X[0].max()

11

In [32]:
y = df.iloc[:, -1].values

In [33]:
y.shape

(10000,)

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [35]:
X_train.shape

(8000, 36350)

In [36]:
X_test.shape

(2000, 36350)

In [37]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [38]:
clf1 = GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()

In [39]:
clf1.fit(X_train, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)

BernoulliNB()

In [40]:
y_pred1 = clf1.predict(X_test)
y_pred2 = clf2.predict(X_test)
y_pred3 = clf3.predict(X_test)

In [41]:
from sklearn.metrics import accuracy_score

In [43]:
print(f"Gaussian: {accuracy_score(y_test, y_pred1)}")
print(f"Multinomial: {accuracy_score(y_test, y_pred2)}")
print(f"Bernoulli: {accuracy_score(y_test, y_pred3)}")

Gaussian: 0.6325
Multinomial: 0.844
Bernoulli: 0.8185


<br>

In [52]:
cv1000 = CountVectorizer(max_features=500)

In [53]:
X = cv1000.fit_transform(df.review).toarray()

In [54]:
X.shape

(10000, 500)

In [55]:
y = df.iloc[:, -1].values

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [57]:
X_train.shape

(8000, 500)

In [58]:
clf1.fit(X_train, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)

y_pred1 = clf1.predict(X_test)
y_pred2 = clf2.predict(X_test)
y_pred3 = clf3.predict(X_test)

In [59]:
print(f"Gaussian: {accuracy_score(y_test, y_pred1)}")
print(f"Multinomial: {accuracy_score(y_test, y_pred2)}")
print(f"Bernoulli: {accuracy_score(y_test, y_pred3)}")

Gaussian: 0.8055
Multinomial: 0.828
Bernoulli: 0.832
